In [1]:
!ls

codes  tf-gpu-env  torch-gpu-env


In [2]:
import os

PATH_CODE = "/mnt/c/Users/User/Documents/codes/DL/deeplearning_codes"
PATH_DATASET = "/mnt/c/Users/User/Documents/DATASETS/human_activities/WISDM_ar_v1.1/"

os.chdir(PATH_CODE)
!ls

MNIST_Ejm_GPU.ipynb	  test_camera.ipynb
README.md		  test_show_video.py
V_1.mp4			  test_show_video_model.py
convert_model_to_h.ipynb  tf_gpu_test1.ipynb
images			  tutorial1_torch.ipynb
lstm1.ipynb		  violence-detection-classifier.ipynb
modelo.h		  violence_detection_classifier2.ipynb
modelo.tflite		  violence_detection_model.pth
output_video.gif	  violence_detection_model_state_dict.pth
output_video.mp4


## Leer Dataset

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv(PATH_DATASET+"WISDM_ar_v1.1_raw.csv", names=["user", "activity", "timestamp", "x-axis", "y-axis", "z-axis"])
df.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164
